<a href="https://colab.research.google.com/github/aruaru0/colab_notebook/blob/main/baseline_%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Aug 14 00:26:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!tar xvf /content/drive/MyDrive/datas/data/Label.tar

label/
label/val_finetune.txt
label/train_finetune.txt


In [3]:
!tar xf /content/drive/MyDrive/datas/data/Val.tar

# restart

In [4]:
!pip install timm

     |████████████████████████████████| 376 kB 8.4 MB/s 


In [5]:
!pip install optuna

     |████████████████████████████████| 302 kB 7.6 MB/s 
     |████████████████████████████████| 80 kB 10.5 MB/s 
     |████████████████████████████████| 164 kB 71.8 MB/s 
     |████████████████████████████████| 75 kB 5.9 MB/s 
     |████████████████████████████████| 49 kB 6.9 MB/s 
     |████████████████████████████████| 111 kB 75.4 MB/s 
     |████████████████████████████████| 141 kB 70.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=4bbd6b70cc251a4eb2f170d480cb7009e872260d81d9328f2788982e88369029
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [6]:
import numpy as np
import pandas as pd
import timm
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from PIL import Image
from torchvision import transforms
import torchvision

import tqdm

import optuna

from PIL import Image

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #(1)
print(device)

cuda:0


In [8]:
class LFFdataset(Dataset):
    def __init__(self, path, df, transform = None):
        self.transform = transform
        self.df = df
        self.path = path
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file = self.path + self.df['file'][idx] 
        #画像読み込み。
        img = Image.open(file)

        #transform事前処理実施
        if self.transform is not None:
            img = self.transform(img)
        label=0 #self.df['class'][idx]

        return img,label
 
class LFFmodel(nn.Module):
  global CONFIG
  def __init__(self):
      super(LFFmodel, self).__init__()
      
      self.net = timm.create_model(CONFIG.base_model, pretrained=True)

  def forward(self, x):
      x = self.net(x)
      return x

# model1 efficientnet_b7

In [ ]:
class CONFIG :
  VALID = '/content/label/val_finetune.txt'
  num_of_worker = 2
  base_model = 'tf_efficientnet_b7_ns'
  batch_size = 4

In [ ]:
df_valid = pd.read_table(CONFIG.VALID, sep=' ', header=None, names=['file','class'])

size = 320
transform_valid = transforms.Compose([transforms.Resize((size, size)), transforms.ToTensor()])

dataset_valid = LFFdataset('./Val/', df_valid, transform_valid)
dataloader = DataLoader(dataset_valid, batch_size=CONFIG.batch_size, shuffle=False, num_workers=CONFIG.num_of_worker)


model = LFFmodel().to(device)
PATH = '/content/drive/MyDrive/datas/last.pth'
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [39]:
# DataLoaderからデータをバッチごとに取り出す
N = len(df_valid)//CONFIG.batch_size

model.eval()
pred = np.array([])

for inputs, labels in tqdm.tqdm(dataloader, total = N) :
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(inputs)
      outputs = outputs.detach().cpu().numpy()
      if len(pred) == 0 :
        pred = outputs
      else :
        pred = np.concatenate([pred, outputs], axis=0)


13366it [25:32,  8.72it/s]


In [40]:
pred1 = pred

# efficientnet_b7 TTA

In [41]:
df_valid = pd.read_table(CONFIG.VALID, sep=' ', header=None, names=['file','class'])

size = 320
transform_valid = transforms.Compose([transforms.CenterCrop(size), transforms.ToTensor()])

dataset_valid = LFFdataset('./Val/', df_valid, transform_valid)
dataloader = DataLoader(dataset_valid, batch_size=CONFIG.batch_size, shuffle=False, num_workers=CONFIG.num_of_worker)


model = LFFmodel().to(device)
PATH = '/content/drive/MyDrive/datas/last.pth'
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [42]:
# DataLoaderからデータをバッチごとに取り出す
N = len(df_valid)//CONFIG.batch_size

model.eval()
pred = np.array([])

for inputs, labels in tqdm.tqdm(dataloader, total = N) :
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(inputs)
      outputs = outputs.detach().cpu().numpy()
      if len(pred) == 0 :
        pred = outputs
      else :
        pred = np.concatenate([pred, outputs], axis=0)


13366it [25:21,  8.79it/s]


In [ ]:
pred2 = pred

# efficientnet b7 flip

In [44]:
df_valid = pd.read_table(CONFIG.VALID, sep=' ', header=None, names=['file','class'])

size = 320
transform_valid = transforms.Compose([transforms.CenterCrop(size), transforms.ToTensor()])

dataset_valid = LFFdataset('./Val/', df_valid, transform_valid)
dataloader = DataLoader(dataset_valid, batch_size=CONFIG.batch_size, shuffle=False, num_workers=CONFIG.num_of_worker)


model = LFFmodel().to(device)
PATH = '/content/drive/MyDrive/datas/last.pth'
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [45]:
# DataLoaderからデータをバッチごとに取り出す
N = len(df_valid)//CONFIG.batch_size

model.eval()
pred = np.array([])

for inputs, labels in tqdm.tqdm(dataloader, total = N) :
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(inputs)
      outputs = outputs.detach().cpu().numpy()
      if len(pred) == 0 :
        pred = outputs
      else :
        pred = np.concatenate([pred, outputs], axis=0)


13366it [25:16,  8.81it/s]


In [46]:
pred3 = pred

# resnetRS152

In [47]:
class CONFIG :
  VALID = '/content/label/val_finetune.txt'
  num_of_worker = 2
  base_model = 'resnetrs152'
  batch_size = 4

In [48]:
df_valid = pd.read_table(CONFIG.VALID, sep=' ', header=None, names=['file','class'])

size = 320
transform_valid = transforms.Compose([transforms.Resize((size, size)), transforms.ToTensor()])

dataset_valid = LFFdataset('./Val/', df_valid, transform_valid)
dataloader = DataLoader(dataset_valid, batch_size=CONFIG.batch_size, shuffle=False, num_workers=CONFIG.num_of_worker)


model = LFFmodel().to(device)
PATH = '/content/drive/MyDrive/datas/last_resnetrs152.pth'
checkpoint = torch.load(PATH)
#model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [49]:
# DataLoaderからデータをバッチごとに取り出す
N = len(df_valid)//CONFIG.batch_size

model.eval()
pred = np.array([])

for inputs, labels in tqdm.tqdm(dataloader, total = N) :
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(inputs)
      outputs = outputs.detach().cpu().numpy()
      if len(pred) == 0 :
        pred = outputs
      else :
        pred = np.concatenate([pred, outputs], axis=0)


13366it [22:44,  9.79it/s]


In [50]:
pred4 = pred

# resnet152 tta

In [51]:
df_valid = pd.read_table(CONFIG.VALID, sep=' ', header=None, names=['file','class'])

size = 320
transform_valid = transforms.Compose([transforms.CenterCrop(size), transforms.ToTensor()])

dataset_valid = LFFdataset('./Val/', df_valid, transform_valid)
dataloader = DataLoader(dataset_valid, batch_size=CONFIG.batch_size, shuffle=False, num_workers=CONFIG.num_of_worker)

model = LFFmodel().to(device)
PATH = '/content/drive/MyDrive/datas/last_resnetrs152.pth'
checkpoint = torch.load(PATH)
#model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [52]:
# DataLoaderからデータをバッチごとに取り出す
N = len(df_valid)//CONFIG.batch_size

model.eval()
pred = np.array([])

for inputs, labels in tqdm.tqdm(dataloader, total = N) :
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(inputs)
      outputs = outputs.detach().cpu().numpy()
      if len(pred) == 0 :
        pred = outputs
      else :
        pred = np.concatenate([pred, outputs], axis=0)


13366it [23:08,  9.63it/s]


In [53]:
pred5 = pred

# resnetrs 152 - horizontal flip

In [54]:
df_valid = pd.read_table(CONFIG.VALID, sep=' ', header=None, names=['file','class'])

size = 320
transform_valid = transforms.Compose([transforms.Resize((size, size)), 
                                      transforms.ToTensor(),
                                      torchvision.transforms.RandomHorizontalFlip(p=1.0)])

dataset_valid = LFFdataset('./Val/', df_valid, transform_valid)
dataloader = DataLoader(dataset_valid, batch_size=CONFIG.batch_size, shuffle=False, num_workers=CONFIG.num_of_worker)


model = LFFmodel().to(device)
PATH = '/content/drive/MyDrive/datas/last_resnetrs152.pth'
checkpoint = torch.load(PATH)
#model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [55]:
# DataLoaderからデータをバッチごとに取り出す
N = len(df_valid)//CONFIG.batch_size

model.eval()
pred = np.array([])

for inputs, labels in tqdm.tqdm(dataloader, total = N) :
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(inputs)
      outputs = outputs.detach().cpu().numpy()
      if len(pred) == 0 :
        pred = outputs
      else :
        pred = np.concatenate([pred, outputs], axis=0)


13366it [22:50,  9.75it/s]


In [56]:
pred6 = pred

# resnetrs50

In [28]:
class CONFIG :
  VALID = '/content/label/val_finetune.txt'
  num_of_worker = 2
  base_model = 'resnetrs50'
  batch_size = 8

In [39]:
df_valid = pd.read_table(CONFIG.VALID, sep=' ', header=None, names=['file','class'])

size = 320
transform_valid = transforms.Compose([
                                      transforms.Resize((size, size)), 
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225]
                                      ),
                                      ])

dataset_valid = LFFdataset('./Val/', df_valid, transform_valid)
dataloader = DataLoader(dataset_valid, batch_size=CONFIG.batch_size, shuffle=False, num_workers=CONFIG.num_of_worker)


model = LFFmodel().to(device)
PATH = '/content/drive/MyDrive/datas/last_resnetrs50.pth'
checkpoint = torch.load(PATH)
#model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [40]:
# DataLoaderからデータをバッチごとに取り出す
N = len(df_valid)//CONFIG.batch_size

model.eval()
pred = np.array([])

for inputs, labels in tqdm.tqdm(dataloader, total = N) :
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(inputs)
      outputs = outputs.detach().cpu().numpy()
      if len(pred) == 0 :
        pred = outputs
      else :
        pred = np.concatenate([pred, outputs], axis=0)


6683it [10:15, 10.87it/s]


In [41]:
pred7 = pred

# resnet50 tta

In [44]:
df_valid = pd.read_table(CONFIG.VALID, sep=' ', header=None, names=['file','class'])

size = 320
transform_valid = transforms.Compose([
                                      transforms.CenterCrop(size), 
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225]
                                      ),
                                      ])

dataset_valid = LFFdataset('./Val/', df_valid, transform_valid)
dataloader = DataLoader(dataset_valid, batch_size=CONFIG.batch_size, shuffle=False, num_workers=CONFIG.num_of_worker)

model = LFFmodel().to(device)
PATH = '/content/drive/MyDrive/datas/last_resnetrs50.pth'
checkpoint = torch.load(PATH)
#model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [45]:
# DataLoaderからデータをバッチごとに取り出す
N = len(df_valid)//CONFIG.batch_size

model.eval()
pred = np.array([])

for inputs, labels in tqdm.tqdm(dataloader, total = N) :
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(inputs)
      outputs = outputs.detach().cpu().numpy()
      if len(pred) == 0 :
        pred = outputs
      else :
        pred = np.concatenate([pred, outputs], axis=0)


6683it [09:33, 11.66it/s]


In [46]:
pred8 = pred

# resnet 270

In [11]:
class CONFIG :
  VALID = '/content/label/val_finetune.txt'
  num_of_worker = 2
  base_model = 'resnetrs270'
  batch_size = 8

In [12]:
df_valid = pd.read_table(CONFIG.VALID, sep=' ', header=None, names=['file','class'])

size = 320
transform_valid = transforms.Compose([
                                      transforms.Resize((size, size)), 
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225]
                                      ),
                                      ])

dataset_valid = LFFdataset('./Val/', df_valid, transform_valid)
dataloader = DataLoader(dataset_valid, batch_size=CONFIG.batch_size, shuffle=False, num_workers=CONFIG.num_of_worker)


model = LFFmodel().to(device)
PATH = '/content/drive/MyDrive/datas/best_resnetrs270.pth'
checkpoint = torch.load(PATH)
#model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
model.load_state_dict(checkpoint['model'])

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rs-weights/resnetrs270_ema-b40e674c.pth" to /root/.cache/torch/hub/checkpoints/resnetrs270_ema-b40e674c.pth


<All keys matched successfully>

In [13]:
# DataLoaderからデータをバッチごとに取り出す
N = len(df_valid)//CONFIG.batch_size

model.eval()
pred = np.array([])

for inputs, labels in tqdm.tqdm(dataloader, total = N) :
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(inputs)
      outputs = outputs.detach().cpu().numpy()
      if len(pred) == 0 :
        pred = outputs
      else :
        pred = np.concatenate([pred, outputs], axis=0)


6683it [22:21,  4.98it/s]


In [14]:
pred9 = pred

# resnet270 tta

In [20]:
df_valid = pd.read_table(CONFIG.VALID, sep=' ', header=None, names=['file','class'])

size = 320
transform_valid = transforms.Compose([
                                      transforms.CenterCrop(size), 
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225]
                                      ),
                                      ])

dataset_valid = LFFdataset('./Val/', df_valid, transform_valid)
dataloader = DataLoader(dataset_valid, batch_size=CONFIG.batch_size, shuffle=False, num_workers=CONFIG.num_of_worker)


model = LFFmodel().to(device)
PATH = '/content/drive/MyDrive/datas/best_resnetrs270.pth'
checkpoint = torch.load(PATH)
#model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [21]:
# DataLoaderからデータをバッチごとに取り出す
N = len(df_valid)//CONFIG.batch_size

model.eval()
pred = np.array([])

for inputs, labels in tqdm.tqdm(dataloader, total = N) :
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(inputs)
      outputs = outputs.detach().cpu().numpy()
      if len(pred) == 0 :
        pred = outputs
      else :
        pred = np.concatenate([pred, outputs], axis=0)


6683it [22:22,  4.98it/s]


In [22]:
pred10 = pred

# アンサンブル optuna

In [25]:
def objective(trial):
    x1 = trial.suggest_uniform("x1", 0, 1)
    x2 = trial.suggest_uniform("x2", 0, 1)
    x3 = trial.suggest_uniform("x3", 0, 1)
    x4 = trial.suggest_uniform("x4", 0, 1)
    x5 = trial.suggest_uniform("x5", 0, 1)
    x6 = trial.suggest_uniform("x6", 0, 1)
    x7 = trial.suggest_uniform("x7", 0, 1)
    x8 = trial.suggest_uniform("x8", 0, 1)
    x9 = trial.suggest_uniform("x9", 0, 1)
    x10 = trial.suggest_uniform("x10", 0, 1)
    tot = x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10
    pred = x1*pred1 + x2*pred2 + x3*pred3 + x4*pred4 + x5*pred5 + x6*pred6 + x7*pred7 + x8*pred8 + x9*pred9 + x10*pred10
    #ret = pred / tot
    df = df_valid
    df['pred1'] = np.argmax(pred, axis=1) 
    ret = sum(df['class']==df['pred1'])/len(df)
    return ret

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# 探索後の最良値
print(study.best_value) 
print(study.best_params) 


[I 2021-08-14 01:56:47,553] A new study created in memory with name: no-name-2f60e6b4-3b98-46a1-8086-3c80824a96d5
[I 2021-08-14 01:56:48,667] Trial 0 finished with value: 0.8859937899816692 and parameters: {'x1': 0.5977429767181637, 'x2': 0.4228889927629924, 'x3': 0.4907897135467515, 'x4': 0.3168823169550675, 'x5': 0.4579734957983038, 'x6': 0.6576673006233527, 'x7': 0.22955340113243794, 'x8': 0.43937113836502173, 'x9': 0.21823235165829968, 'x10': 0.5956208212509834}. Best is trial 0 with value: 0.8859937899816692.
[I 2021-08-14 01:56:49,580] Trial 1 finished with value: 0.8842355317795818 and parameters: {'x1': 0.6652797213423173, 'x2': 0.1933457571739935, 'x3': 0.2363490736036995, 'x4': 0.8996700198064049, 'x5': 0.3996415095680472, 'x6': 0.8825829180475909, 'x7': 0.9046352365437823, 'x8': 0.45184048669338195, 'x9': 0.8059821206938713, 'x10': 0.06976622216859096}. Best is trial 0 with value: 0.8859937899816692.
[I 2021-08-14 01:56:50,483] Trial 2 finished with value: 0.880662900751936 

0.8876959335602859
{'x1': 0.8042727675822595, 'x2': 0.5317776493054536, 'x3': 0.5670412768073269, 'x4': 0.649086896709438, 'x5': 0.8752888940008074, 'x6': 0.5573150385007948, 'x7': 0.7717965497804117, 'x8': 0.4801557161845649, 'x9': 0.6965793453602516, 'x10': 0.9530145237044476}


# 単体チェック

In [23]:
df = df_valid
df['pred1'] = np.argmax(pred, axis=1)

In [24]:
sum(df['class']==df['pred1'])/len(df)

0.8316748344618607

In [62]:
# 0.8208260072574913 org b7 320
# 0.8299165762597733 tta b7 320
# 0.8460401780704051 mix b7 320
# 0.8481538288878082 org res152 320
# 0.8459653585724439 tta res152 320
# 0.8623134188769593 mix res152 320
# 0.8714601025027122 mix b7 b7tta res152 320
# 0.8793909692865961 mix b7 b7tta res152 res152tta 320
# 0.8796902472784408 min b7 b7tta res152 res152tta res152flip 320
# 0.8801765740151883
# 0.880475852007033
# 0.8817103737233923
# 0.8818974224682953
# 0.8849837267591935
# 0.8876959335602859

In [50]:
import pickle
with open('/content/drive/MyDrive/datas/dump.pkl', 'wb') as wr:
  pickle.dump(pred1, wr)
  pickle.dump(pred2, wr)
  pickle.dump(pred3, wr)
  pickle.dump(pred4, wr)
  pickle.dump(pred5, wr)
  pickle.dump(pred6, wr)
  pickle.dump(pred7, wr)
  pickle.dump(pred8, wr)

In [17]:
import pickle
with open('/content/drive/MyDrive/datas/dump.pkl', 'rb') as rd:
  pred1 = pickle.load(rd)
  pred2 = pickle.load(rd)
  pred3 = pickle.load(rd)
  pred4 = pickle.load(rd)
  pred5 = pickle.load(rd)
  pred6 = pickle.load(rd)
  pred7 = pickle.load(rd)
  pred8 = pickle.load(rd)
